# 使用_slots_

当我们定义了一个class，创建了一个class的实例后，我们可以给该实例绑定任何属性和方法，这就是动态语言的灵活性

In [1]:
class Student(object):
    pass

In [3]:
s = Student()
s.name = 'mike'
print(s.name)

mike


In [6]:
def set_age(self,age):
    self.age = age
from types import MethodType
s.set_age = MethodType(set_age, s)
s.set_age(25)
s.age

25

Python允许在定义class的时候，定义一个特殊的__slots__变量，来限制该class实例能添加的属性

In [7]:
class Student(object):
    __slots__ = ('name', 'age') # 用tuple定义允许绑定的属性名称

In [8]:
s = Student() # 创建新的实例
s.name = 'Michael' # 绑定属性'name'

In [9]:
s.age = 25 # 绑定属性'age'
s.score = 99 # 绑定属性'score'

AttributeError: 'Student' object has no attribute 'score'

使用__slots__要注意，__slots__定义的属性仅对当前类实例起作用，对继承的子类是不起作用的。除非在子类中也定义__slots__，这样，子类实例允许定义的属性就是自身的__slots__加上父类的__slots__

# 使用@property

把一个getter方法变成属性，只需要加上@property就可以了，此时，@property本身又创建了另一个装饰器@score.setter，负责把一个setter方法变成属性赋值，于是，我们就拥有一个可控的属性操作

In [10]:
class Student(object):

    @property
    def score(self):
        return self._score

    @score.setter
    def score(self, value):
        if not isinstance(value, int):
            raise ValueError('score must be an integer!')
        if value < 0 or value > 100:
            raise ValueError('score must between 0 ~ 100!')
        self._score = value

In [13]:
s = Student()
s.score = 60
s.score

60

In [15]:
s.score = 99
s.score

99

In [16]:
s.score = 120
s.score

ValueError: score must between 0 ~ 100!

还可以定义只读属性，只定义getter方法，不定义setter方法就是一个只读属性

In [17]:
class Student(object):

    @property
    def birth(self):
        return self._birth

    @birth.setter
    def birth(self, value):
        self._birth = value

    @property
    def age(self):
        return 2015 - self._birth

# 多重继承

In [20]:
class Animal(object):
    pass

# 大类:
class Mammal(Animal):
    def milk(self):
        print('need milk')
    pass

class Bird(Animal):
    pass

# 各种动物:
class Dog(Mammal):
    pass

class Bat(Mammal):
    pass

class Parrot(Bird):
    pass

class Ostrich(Bird):
    pass

In [21]:
class Runnable(object):
    def run(self):
        print('Running...')

class Flyable(object):
    def fly(self):
        print('Flying...')

In [22]:
class Dog(Mammal, Runnable):
    pass

In [24]:
dog = Dog()
dog.milk()

need milk


In [25]:
dog.run()

Running...


# 类对象与实例对象

类对象就是可以用类名字直接使用表示的对象，它支持两种操作，直接属性使用和实例化。对于类属性的使用，直接使用类名.属性即可。对于类方法的使用，需要实例化一个对象后，将对象名赋值给self使用

In [41]:
class test:
    data = 1
    def __init__(self):
        self.property=12

    def test2(self):
        print('hello')
        return 0
t = test()
print(test.data)
print(t.data)
print(test.test2)
print(t.test2())
print(test.test2(t))
print('*'*80)

print(t.property)
print(test.property)


1
1
<function test.test2 at 0x0000000005064AE8>
hello
0
hello
0
********************************************************************************
12


AttributeError: type object 'test' has no attribute 'property'

1、作为test的类对象的变量 （data），每次创建一个新的实例对象，类对象变量就多一个引用指向它，通过实例对象来修改类对象的变量的取值，实际上是让实例对象的data指向了另外一块内存变量。实例对象是类对象的一个拷贝。   2、可以通过实例对象._class_.data 来获取类对象的data值，改变类对象的变量的值后，相应实例的值也会发生变化。类对象的变量在实例中实际上是只读的，任何实例都无法修改类对象变量的值（test1.data=2 实际上是让实例的变量指向了另一块内存，当再生成一个新的对象时,值仍然还是1），通过实例对象._class_.data可以修改类对象的属性值

# 定制类(回头再看)

这些特殊的函数或者变量，前后都是双下划线，注意了

__str__()、__repr__()，两者的区别是__str__()返回用户看到的字符串，而__repr__()返回程序开发者看到的字符串，也就是说，__repr__()是为调试服务的。

In [45]:
class Student(object):
    def __init__(self,name):
        self.name = name
print(Student('mike'))

In [47]:
class Student(object):
    def __init__(self,name):
        self.name = name
    def __str__(self):
        return 'student object(name:%s)' %self.name
print(Student('mike'))

student object(name:mike)


In [49]:
class Student(object):
    def __init__(self,name):
        self.name = name
    def __str__(self):
        return 'student object(name:%s)' %self.name
s = Student('mike')
s

In [51]:
class Student(object):
    def __init__(self,name):
        self.name = name
    def __str__(self):
        return 'student object(name:%s)' %self.name
    __repr__ = __str__
    
s = Student('mike')
s

student object(name:mike)

如果一个类想被用于for ... in循环，类似list或tuple那样，就必须实现一个__iter__()方法，该方法返回一个迭代对象，然后，Python的for循环就会不断调用该迭代对象的__next__()方法拿到循环的下一个值，直到遇到StopIteration错误时退出循环。

In [52]:
class Fib(object):
    def __init__(self):
        self.a, self.b = 0, 1 # 初始化两个计数器a，b

    def __iter__(self):
        return self # 实例本身就是迭代对象，故返回自己

    def __next__(self):
        self.a, self.b = self.b, self.a + self.b # 计算下一个值
        if self.a > 100000: # 退出循环的条件
            raise StopIteration()
        return self.a # 返回下一个值

In [53]:
for i in Fib():
    print(i)

1
1
2
3
5
8
13
21
34
55
89
144
233
377
610
987
1597
2584
4181
6765
10946
17711
28657
46368
75025


__getitem__,__setitem__,__delitem__,通过上面的方法，我们自己定义的类表现得和Python自带的list、tuple、dict没什么区别，这完全归功于动态语言的“鸭子类型”，不需要强制继承某个接口。

__getattr__

__call__

# 枚举类

为枚举类型定义一个class类型，然后，每个常量都是class的一个唯一实例。Python提供了Enum类来实现这个功能

In [58]:
from enum import Enum

Month = Enum('Mon', ('Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'))

In [59]:
for name, member in Month.__members__.items():
    print(name, '=>', member, ',', member.value)

Jan => Mon.Jan , 1
Feb => Mon.Feb , 2
Mar => Mon.Mar , 3
Apr => Mon.Apr , 4
May => Mon.May , 5
Jun => Mon.Jun , 6
Jul => Mon.Jul , 7
Aug => Mon.Aug , 8
Sep => Mon.Sep , 9
Oct => Mon.Oct , 10
Nov => Mon.Nov , 11
Dec => Mon.Dec , 12


@unique装饰器可以帮助我们检查保证没有重复值

In [65]:
from enum import Enum, unique

@unique
class Weekday(Enum):
    Sun = 0 # Sun的value被设定为0
    Mon = 9
    Tue = 2
    Wed = 3
    Thu = 4
    Fri = 5
    Sat = 6

In [67]:
print(Weekday.Tue)
print(Weekday.Tue.value)
print(Weekday(9))

Weekday.Tue
2
Weekday.Mon


In [68]:
for name, member in Weekday.__members__.items():
     print(name, '=>', member)


Sun => Weekday.Sun
Mon => Weekday.Mon
Tue => Weekday.Tue
Wed => Weekday.Wed
Thu => Weekday.Thu
Fri => Weekday.Fri
Sat => Weekday.Sat


# 元类

通过type()函数创建的类和直接写class是完全一样的，因为Python解释器遇到class定义时，仅仅是扫描一下class定义的语法，然后调用type()函数创建出class。

要创建一个class对象，type()函数依次传入3个参数：

（1）class的名称；
（2）继承的父类集合，注意Python支持多重继承，如果只有一个父类，别忘了tuple的单元素写法；
（3）class的方法名称与函数绑定，这里我们把函数fn绑定到方法名hello上。

In [69]:
def fn(self, name='world'): # 先定义函数
     print('Hello, %s.' % name)
        
Hello = type('Hello', (object,), dict(hello=fn)) # 创建Hello class

h = Hello()
h.hello()


Hello, world.


In [70]:
print(type(Hello))
print(type(h))

<class 'type'>
<class '__main__.Hello'>


metaclass，直译为元类。先定义metaclass，就可以创建类，最后创建实例。所以，metaclass允许你创建类或者修改类。换句话说，你可以把类看成是metaclass创建出来的“实例”。